#### Advent of Code Day 04
Coded in Db2 SQL. See [readme](https://github.com/ecrooks/aoc2021_SQL) for more details on how to use. 

Env setup - only needs to run once on each machine

In [ ]:
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import modules and load the SQL magic

In [ ]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import getpass
import csv

Connect to the database. Change the values of user, host, and password to match your environment. For connection to a local host, use 'localhost' for the host name. Also change the port number and database name in the connection string.

In [ ]:
user='db2inst1'
host='localhost'
dbname='aocdb'


password = getpass.getpass('Enter password for '+user+' on '+dbname)

%sql db2+ibm_db://$user:$password@$host:50000/$dbname

## Day 3, Part 1 Solution

### Problem statement: 

>--- Day 4: Giant Squid ---
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.  
>  
>Maybe it wants to play bingo?  
>  
>Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)  
>  
>The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:  
>  
>7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1  
> 
```
>22 13 17 11  0  
> 8  2 23  4 24  
>21  9 14 16  7  
> 6 10  3 18  5  
> 1 12 20 15 19  
>  
> 3 15  0  2 22  
> 9 18 13 17  5  
>19  8  7 25 23  
>20 11 10 24  4  
>14 21 16 12  6  
>  
>14 21 17 24  4  
>10 16 15  9 19  
>18  8 23 26 20  
>22 11 13  6  5  
> 2  0 12  3  7  

```
>After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):  
>  
```
>22 13 17 11  0         3 15  0  2 22        14 21 17 24  4  
> 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19  
>21  9 14 16  7        19  8  7 25 23        18  8 23 26 20  
> 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5  
> 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7  
```
>After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:  
>  
```
>22 13 17 11  0         3 15  0  2 22        14 21 17 24  4  
> 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19  
>21  9 14 16  7        19  8  7 25 23        18  8 23 26 20  
> 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5  
> 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7  
```
>Finally, 24 is drawn:  
```
>22 13 17 11  0         3 15  0  2 22        14 21 17 24  4  
> 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19  
>21  9 14 16  7        19  8  7 25 23        18  8 23 26 20  
> 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5  
> 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7  
```
>At this point, the third board wins because it has at least one complete row or column of marked numbers (in this >case, the entire top row is marked: 14 21 17 24 4).  
>  
>The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that >board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, >24, to get the final score, 188 * 24 = 4512.  
>  
>To guarantee victory against the giant squid, figure out which board will win first. What will your final score be >if you choose that board?  

### Create Objects and Load Data

Table for numbers chosen

In [ ]:
%sql drop table aoc_day04_picker

In [ ]:
%%sql 
create table if not exists aoc_day04_picker (
	seq int generated by default as identity primary key
	, num_selected int not null
	, processed smallint not null default 0
);

Table for game boards

In [ ]:
%sql drop table aoc_day04_boards

In [ ]:
%%sql 
create table if not exists aoc_day04_boards (
	seq int generated by default as identity primary key
	, game_num int not null
	, row_num smallint not null
    , col_num smallint not null
    , value int not null
    , picked smallint not null default 0
);
create unique index ix_aoc_day04_boards_position on aoc_day04_boards (game_num, row_num, col_num) include (picked) allow reverse scans;
create index ix_aoc_day04_boards_value on aoc_day04_boards ( value ) allow reverse scans;

Data is not in a format that is easy for us to parse it with Db2 alone, so we'll parse it with Python first, then load it in.

In [ ]:
line_num=0
game_num=-1
horiz_pos=0
vert_pos=-1
with open("data/day04_input.txt") as fin, open("data/day04_input.csv", 'w') as fout, open("data/day04_input_bp.csv", 'w') as pfout:
    o=csv.writer(fout)
    op=csv.writer(pfout)
    for line in fin:
        #display(line_num)
        if line_num == 0 :
            #display(line)
            line = line[:-1]
            #display(line)
            #display(type(line))
            picked_vals=list(line.split(","))
            #display(picked_vals)
            #op.writerows([picked_vals])
            for picked_val in picked_vals:
                display
                op.writerow([picked_val])
            line_num = line_num + 1
            continue
        if line == '\n' :
            game_num = game_num + 1
            vert_pos=-1
        #display("game_num: "+str(game_num))
        horiz_pos=0
        for val in line.split():
                #display([game_num, vert_pos, horiz_pos, val])
                o.writerow([game_num, vert_pos, horiz_pos, val])
                horiz_pos = horiz_pos + 1
        vert_pos = vert_pos + 1
        line_num = line_num + 1

Import data into the table. Placing the file in the repo data directory will make it available in the right location if you use the container provided.

In [ ]:
%%sql 
delete from aoc_day04_picker;
call admin_cmd ('import from /database/day04_input_bp.csv of del 
    insert into aoc_day04_picker (num_selected)');

In [ ]:
%%sql 
delete from aoc_day04_boards;
call admin_cmd ('import from /database/day04_input.csv of del 
    insert into aoc_day04_boards (game_num, row_num, col_num, value)');

Quick verification

In [ ]:
%sql select count(*) from aoc_day04_picker;

In [ ]:
%sql select count(*) from aoc_day04_boards;

In [ ]:
%sql select * from aoc_day04_picker fetch first 5 rows only;

In [ ]:
%sql select * from aoc_day04_boards fetch first 5 rows only;

In [ ]:
%%sql update aoc_day04_boards as b 
    set picked=(select seq from aoc_day04_picker p where b.value=p.num_selected)

In [ ]:
%sql select * from aoc_day04_boards fetch first 5 rows only;

In [ ]:
%sql select * from aoc_day04_boards where picked=0

### Solution to Part 1

Find lowest winning row

In [ ]:
%%sql with rowsum as (select game_num, row_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, row_num
having count(picked)=5)
select game_num, max_seq from rowsum order by max_seq fetch first 1 row only 

Find lowest winning column

In [ ]:
%%sql with colsum as (select game_num, col_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, col_num
having count(picked)=5)
select game_num, max_seq from colsum order by max_seq fetch first 1 row only 

Calculate board score

In [ ]:
%%sql select sum(value) * (select value from aoc_day04_boards where game_num=2 and picked=12)
from aoc_day04_boards
where game_num = 2
    and picked > 12

Put it all together in one query

In [ ]:
%%sql with rowsum as (select game_num, row_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, row_num
having count(picked)=5),
colsum as (select game_num, col_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, col_num
having count(picked)=5),
allwin as (select game_num, max_seq from rowsum where max_seq=(select min(max_seq) from rowsum) union all select game_num, max_seq from colsum where max_seq=(select min(max_seq) from colsum) ),
finalwin as (select * from allwin where max_seq = (select min(max_seq) from allwin))
select sum(value) * (select value from aoc_day04_boards where game_num=(select game_num from finalwin) and picked=(select max_seq from finalwin))
from aoc_day04_boards
where game_num = (select game_num from finalwin)
    and picked > (select max_seq from finalwin)

## Day 4, Part 2 Solution

### Problem statement: 
>--- Part Two ---  
>On the other hand, it might be wise to try a different strategy: let the giant squid win.  
>
>You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.
>
>In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.
>
>Figure out which board will win last. Once it wins, what would its final score be?

### Final Solution to Part 2 

In [ ]:
%%sql with rowsum as (select game_num, row_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, row_num
having count(picked)=5)
select * 
from rowsum as a
where a.max_seq=(select min(b.max_seq) 
                 from rowsum as b 
                 where a.game_num=b.game_num  
                 group by game_num)

In [ ]:
%%sql with colsum as (select game_num, col_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, col_num
having count(picked)=5)
select * 
from colsum as a
where a.max_seq=(select min(b.max_seq) 
                 from colsum as b 
                 where a.game_num=b.game_num  
                 group by game_num)

Putting it together

In [ ]:
%%sql with rowsum as (select game_num, row_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, row_num
having count(picked)=5),
colsum as (select game_num, col_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, col_num
having count(picked)=5),
allwin as (select game_num, max_seq from rowsum as a where max_seq=(select min(b.max_seq) 
                 from rowsum as b 
                 where a.game_num=b.game_num  
                 group by game_num) 
           union 
               select game_num, max_seq from colsum as c where max_seq=(select min(d.max_seq) 
                 from colsum as d 
                 where c.game_num=d.game_num  
                 group by game_num) ),
eachwin as (select * from allwin as aw where max_seq = (select min(max_seq) from allwin as aw2 where aw.game_num=aw2.game_num)),
finalwin as (select * from eachwin where max_seq=(select max(max_seq) from eachwin))
select * from finalwin order by game_num

Adding the score calcuation

In [ ]:
%%sql with rowsum as (select game_num, row_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, row_num
having count(picked)=5),
colsum as (select game_num, col_num, count(picked) as count, max(picked) as max_seq
from aoc_day04_boards 
where picked !=0 
group by game_num, col_num
having count(picked)=5),
allwin as (select game_num, max_seq from rowsum as a where max_seq=(select min(b.max_seq) 
                 from rowsum as b 
                 where a.game_num=b.game_num  
                 group by game_num) 
           union 
               select game_num, max_seq from colsum as c where max_seq=(select min(d.max_seq) 
                 from colsum as d 
                 where c.game_num=d.game_num  
                 group by game_num) ),
eachwin as (select * from allwin as aw where max_seq = (select min(max_seq) from allwin as aw2 where aw.game_num=aw2.game_num)),
finalwin as (select * from eachwin where max_seq=(select max(max_seq) from eachwin))
select sum(value) * (select value from aoc_day04_boards where game_num=(select min(game_num) from finalwin) and picked=(select max_seq from finalwin))
from aoc_day04_boards
where game_num = (select game_num from finalwin)
    and picked > (select max_seq from finalwin)